In [9]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import models
from torch import nn, optimc

In [10]:
# Veri yolları
import os

base_dir = r"C:\Users\DeLL\Desktop\flowers"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")
test_dir = os.path.join(base_dir, "test")

print("Train Directory Exists:", os.path.exists(train_dir))
print("Valid Directory Exists:", os.path.exists(valid_dir))
print("Test Directory Exists:", os.path.exists(test_dir))
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")
test_dir = os.path.join(base_dir, "test")

# Veri dönüşümleri
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Görselleri 224x224 boyutlarına getir
    transforms.ToTensor(),  # Tensöre dönüştür
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalleştirme
])


Train Directory Exists: True
Valid Directory Exists: True
Test Directory Exists: True


In [11]:
# Veri setlerini yükle
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Veri yükleyiciler
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Sınıf isimlerini çıkar
classes = train_dataset.classes
print("Sınıflar:", classes)

Sınıflar: ['Abutilon Çiçeği', 'Abuzambak', 'Acemi Borusu Bitkisi', 'Aconitum Fischeri', 'Afrika Papatyası', 'Ahşap Anemon Çiçeği', 'Akasma', 'Alev Çiçeği', 'Altın Otu', 'Anemon Çiçeği', 'Antoryum', 'Aslanağzı', 'Atatürk Çiçeği', 'Ateş Lalesi', 'Ay Orkidesi', 'Aynısefa Çiçeği', 'Ayyıldız Çiçeği', 'Ayçiçeği', 'Açelya', 'Balon Çiçeği', 'Bataklık Süseni', 'Begonvil', 'Bergamot', 'Beyaz Gaura Bitkisi', 'Bodrum Papatyası', 'Brovalya Çiçeği', 'California Haşhaş', 'Cattleya Orkidesi', 'Cautleya Zambak', 'Cennet Kuşu Çiçeği', 'Dağ Çiçeği', 'Deve Dikeni', 'Düğün Çiçeği', 'Ermeni Sümbülü', 'Ezan Çiçeği', 'Gazanya Çiçeği', 'Gelincik', 'Gerbera Çiçeği', 'Glayör Çiçeği', 'Guzmanya Çiçeği', 'Gül', 'Gümüş Çalısı', 'GümüşTerlik Orkidesi', 'Gündüz Sefası Çiçeği', 'Güzel Hatun Çiçeği', 'Haseki Küpesi Çiçeği', 'Hava Çiçeği', 'Havaifişek Çiçeği', 'Hercai Menekşe', 'Hint Lotusu', 'Hint Mabet Ağacı', 'Horoz İbiği Çiçeği', 'Itırşahi Çiçeği', 'Japon Gülü', 'Kadife Çiçeği', 'Kala Çiçeği', 'Kamelya Çiçeği', 'Kap

Modelin Eğitimi

In [12]:
# Önceden eğitilmiş ResNet18 modelini yükle
model = models.resnet18(pretrained=True)

# Modelin son katmanını düzenle
num_classes = len(classes)  # Bitki türü sayısı
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Cihaz ayarı (GPU kullanımı)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Kayıp fonksiyonu ve optimizasyon
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Modeli eğit
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


c:\Users\DeLL\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\DeLL\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\DeLL/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:10<00:00, 4.37MB/s]


Epoch 1/10, Loss: 1.5419, Accuracy: 62.88%
Epoch 2/10, Loss: 0.5005, Accuracy: 86.20%
Epoch 3/10, Loss: 0.2260, Accuracy: 94.08%
Epoch 4/10, Loss: 0.2130, Accuracy: 93.94%
Epoch 5/10, Loss: 0.1252, Accuracy: 96.72%
Epoch 6/10, Loss: 0.1123, Accuracy: 96.99%
Epoch 7/10, Loss: 0.2078, Accuracy: 93.86%
Epoch 8/10, Loss: 0.0957, Accuracy: 97.47%
Epoch 9/10, Loss: 0.0783, Accuracy: 97.71%
Epoch 10/10, Loss: 0.0908, Accuracy: 97.39%


Modeli Değerlendirme

In [ ]:
# Doğrulama fonksiyonu
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

# Validasyon ve test doğruluğu
valid_accuracy = evaluate(model, valid_loader)
test_accuracy = evaluate(model, test_loader)
print(f"Validation Accuracy: {valid_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")


Modelin Kaydedilmesi

In [14]:
torch.save(model.state_dict(), "plant_recognition_model.pth")
print("Model kaydedildi.")


Model kaydedildi.


Eğitilen Model ile Tahmin Yapma

In [16]:
from PIL import Image #görüntü dosyalarını işlemek için

# Görüntü tahmini
def predict_image(image_path, model, classes):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0)  # Batch boyutu ekle
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return classes[preds.item()]

# Örnek tahmin
model.load_state_dict(torch.load("plant_recognition_model.pth"))
test_image_path = "C:\\Users\\DeLL\\Desktop\\flowers\\img2.jpg"
predicted_class = predict_image(test_image_path, model, classes)
print(f"Tahmin Edilen Bitki Türü: {predicted_class}")


Tahmin Edilen Bitki Türü: Açelya


C:\Users\DeLL\AppData\Local\Temp\ipykernel_27588\231298520.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("plant_recognition_model.pth